<a href="https://colab.research.google.com/github/pirri87/GOB_Medio_Amb/blob/main/Expediente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries & Load Drive

In [1]:
# libraries
from numpy import NaN, nan as np
import pandas as pd
import re 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load EXCEL

In [3]:
 xls = pd.ExcelFile('/content/drive/MyDrive/00_Gob_Medio_Amb/Expediente_APCA/20221018  SEGUIMIENTO EXPEDIENTES APCA-excel.xlsx')

# Tratamiento de datos

In [4]:
df = pd.read_excel(xls,0)
rows = len(df)

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell U210 is marked as a date but the serial value 6692043 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


## Función para eleminar combinas y añadir datos

In [5]:
def remove_nan (rows, colum):
  cell_buffer = []
  for row in range(rows):
    cell = df.loc[row, colum]
    cell_buffer.append(cell)
    if str(cell) == 'nan': # No me funcionas lo metedos nan asi lo transforme a str y puse nan
      df.at[row, colum] = cell_buffer[-2] # Añado el penultimo numero que tiene dato
      cell_buffer[-1] = cell_buffer[-2] # Añado el penultimo numero que tiene dato

In [6]:
def add_APCA_format (rows, colum):
  for row in range(rows):
    cell = df.loc[row, colum]
    if len(str(cell)) == 1:
      df.at[row, colum] = 'APCA-00' + str(cell)+ '-' + str(df.loc[row, 'PROV.'])
    elif len(str(cell)) == 2:
      df.at[row, colum] = 'APCA-0' + str(cell)+ '-' + str(df.loc[row, 'PROV.'])
    elif len(str(cell)) == 3:
      df.at[row, colum] = 'APCA-' + str(cell)+ '-' + str(df.loc[row, 'PROV.'])

In [7]:
remove_nan(rows, 'APCA Nº')
remove_nan(rows, 'PROV.')
remove_nan(rows, 'TÉCNICO')

In [8]:
add_APCA_format(rows, 'APCA Nº')

# Quitar formato data.time y solo poner fecha y no las horas

In [9]:
for row in range(rows):
  for colum in df.columns:
    cell =str(df.loc[row, colum])
    if re.findall("00:00:00", cell):
      cell = cell.replace("00:00:00", "")
      df.loc[row, colum] = cell

# Guardar en excel

In [10]:
path = '/content/drive/MyDrive/00_Gob_Medio_Amb/Expediente_APCA/expediente-sin-combinaciones-datatime.xlsx'
with pd.ExcelWriter(path) as writer:
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
  df.to_excel(writer, sheet_name="Exp_apcas", index=False)